### Question

"Find the second heaviest animal in the table from each zoo.  In addition to the animal information, return the zoo name as well"

### SQLAlchemy Exploration

This Notebook doesn't have a direct solution like the `Pandas Solution.ipynb`.  It is more of an exploration about what can be done with SQL and SQLAlchemy.  Some of the sections answer the question, but I'm not sure what the "right" way to do it is.

### Database creation and model imports

SQLAlchemy models are in `utils/models.py` and Pydantic models are in `utils/schemas.py`.  While the SQLAlchemy objects are using Pydantic for `repr`, and we used Pydantic to normalize the data ingest in `Cleaning Data.ipynb`, the Pydantic models are not used directly at all here.

In [1]:
!cat ../utils/models.py

import sqlalchemy as sa
import sqlalchemy.orm
from sqlalchemy.ext.declarative import declarative_base

from . import schemas

# ^^ declarative_base also available from sqlalchemy.orm if you have 1.4+

Base = declarative_base()


class Zoo(Base):
    __tablename__ = "zoos"
    id = sa.Column(sa.Integer, primary_key=True, index=True)
    name = sa.Column(sa.String)
    animals = sa.orm.relationship("Animal", back_populates="zoo", order_by="desc(Animal.weight)")

    def __repr__(self):
        return repr(schemas.Zoo.from_orm(self))


class Animal(Base):
    __tablename__ = "animals"
    id = sa.Column(sa.Integer, primary_key=True, index=True)
    weight = sa.Column(sa.Integer)
    zoo_id = sa.Column(sa.Integer, sa.ForeignKey("zoos.id"))
    zoo = sa.orm.relationship("Zoo", back_populates="animals")

    def __repr__(self):
        return repr(schemas.Animal.from_orm(self))


In [2]:
import sys
sys.path.insert(0, "..")
from utils.models import Animal, Zoo
Animal, Zoo

(utils.models.Animal, utils.models.Zoo)

In [3]:
type(Animal), type(Zoo)

(sqlalchemy.ext.declarative.api.DeclarativeMeta,
 sqlalchemy.ext.declarative.api.DeclarativeMeta)

In [4]:
import sqlalchemy as sa
import sqlalchemy.orm

# echo=True will print out all the sql query statements that are executed
# set echo=False for less verbosity
engine = sa.create_engine("sqlite:///../zoo_animals.db", echo=True)
LocalSession = sa.orm.sessionmaker(bind=engine)
db = LocalSession()
db

### Exploring the database

In [5]:
engine.table_names()

2021-04-30 22:23:19,910 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-04-30 22:23:19,911 INFO sqlalchemy.engine.base.Engine ()
2021-04-30 22:23:19,914 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-04-30 22:23:19,915 INFO sqlalchemy.engine.base.Engine ()
2021-04-30 22:23:19,917 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-04-30 22:23:19,918 INFO sqlalchemy.engine.base.Engine ()


['animals', 'zoos']

In [6]:
# Getting row counts
db.query(Animal).count()

2021-04-30 22:23:20,062 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-04-30 22:23:20,063 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT animals.id AS animals_id, animals.weight AS animals_weight, animals.zoo_id AS animals_zoo_id 
FROM animals) AS anon_1
2021-04-30 22:23:20,064 INFO sqlalchemy.engine.base.Engine ()


9

In [7]:
# Alternative count query
db.query(sa.func.count(Animal.id)).scalar()

2021-04-30 22:23:20,201 INFO sqlalchemy.engine.base.Engine SELECT count(animals.id) AS count_1 
FROM animals
2021-04-30 22:23:20,202 INFO sqlalchemy.engine.base.Engine ()


9

### Models

As seen in the `models.py` earlier, the SQLAlchemy models actually use Pydantic for their `repr` so that they look nicer.  Don't be fooled by the output, they're still SQLAlchemy objects.  They also have lazy-loading relationship attributes.  For instance if you start with one Animal, you can jump to the Zoo it belongs to, and then to all other Animals in that Zoo.

In [8]:
item = db.query(Animal).first()
item

2021-04-30 22:23:20,343 INFO sqlalchemy.engine.base.Engine SELECT animals.id AS animals_id, animals.weight AS animals_weight, animals.zoo_id AS animals_zoo_id 
FROM animals
 LIMIT ? OFFSET ?
2021-04-30 22:23:20,344 INFO sqlalchemy.engine.base.Engine (1, 0)


Animal(id=1117, weight=1000, zoo_id=9)

In [9]:
item.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f8a0146c3d0>,
 'weight': 1000,
 'zoo_id': 9,
 'id': 1117}

In [10]:
item.id

1117

In [11]:
item.zoo

2021-04-30 22:23:20,910 INFO sqlalchemy.engine.base.Engine SELECT zoos.id AS zoos_id, zoos.name AS zoos_name 
FROM zoos 
WHERE zoos.id = ?
2021-04-30 22:23:20,911 INFO sqlalchemy.engine.base.Engine (9,)


Zoo(id=9, name='NY')

In [12]:
item.zoo.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f8a0149f130>,
 'name': 'NY',
 'id': 9}

In [13]:
item.zoo.animals

2021-04-30 22:23:21,277 INFO sqlalchemy.engine.base.Engine SELECT animals.id AS animals_id, animals.weight AS animals_weight, animals.zoo_id AS animals_zoo_id 
FROM animals 
WHERE ? = animals.zoo_id ORDER BY animals.weight DESC
2021-04-30 22:23:21,279 INFO sqlalchemy.engine.base.Engine (9,)


[Animal(id=1117, weight=1000, zoo_id=9),
 Animal(id=1415, weight=695, zoo_id=9),
 Animal(id=1410, weight=690, zoo_id=9)]

### Ordered Relationships

One "cheat" way of answering the original question is by architecting the database towards it.  The `Zoo.animals` relationship is ordered by weight.

In [14]:
items = db.query(Animal).group_by(Animal.zoo_id).all()
items

2021-04-30 22:23:21,519 INFO sqlalchemy.engine.base.Engine SELECT animals.id AS animals_id, animals.weight AS animals_weight, animals.zoo_id AS animals_zoo_id 
FROM animals GROUP BY animals.zoo_id
2021-04-30 22:23:21,521 INFO sqlalchemy.engine.base.Engine ()


[Animal(id=1117, weight=1000, zoo_id=9),
 Animal(id=9821, weight=750, zoo_id=11),
 Animal(id=1411, weight=690, zoo_id=12)]

In [15]:
answers = [item.zoo.animals[1] for item in items]
answers

2021-04-30 22:23:21,796 INFO sqlalchemy.engine.base.Engine SELECT zoos.id AS zoos_id, zoos.name AS zoos_name 
FROM zoos 
WHERE zoos.id = ?
2021-04-30 22:23:21,800 INFO sqlalchemy.engine.base.Engine (11,)
2021-04-30 22:23:21,807 INFO sqlalchemy.engine.base.Engine SELECT animals.id AS animals_id, animals.weight AS animals_weight, animals.zoo_id AS animals_zoo_id 
FROM animals 
WHERE ? = animals.zoo_id ORDER BY animals.weight DESC
2021-04-30 22:23:21,809 INFO sqlalchemy.engine.base.Engine (11,)
2021-04-30 22:23:21,814 INFO sqlalchemy.engine.base.Engine SELECT zoos.id AS zoos_id, zoos.name AS zoos_name 
FROM zoos 
WHERE zoos.id = ?
2021-04-30 22:23:21,817 INFO sqlalchemy.engine.base.Engine (12,)
2021-04-30 22:23:21,819 INFO sqlalchemy.engine.base.Engine SELECT animals.id AS animals_id, animals.weight AS animals_weight, animals.zoo_id AS animals_zoo_id 
FROM animals 
WHERE ? = animals.zoo_id ORDER BY animals.weight DESC
2021-04-30 22:23:21,821 INFO sqlalchemy.engine.base.Engine (12,)


[Animal(id=1415, weight=695, zoo_id=9),
 Animal(id=9821, weight=750, zoo_id=11),
 Animal(id=9992, weight=1040, zoo_id=12)]

In [16]:
for item in answers:
    print("%s (%s): %s (%s)" % (item.zoo.name, item.zoo.id, item.weight, item.id))

NY (9): 695 (1415)
LA (11): 750 (9821)
Detroit (12): 1040 (9992)


### Over / Row Numbering

Instead of using the relations in the database, we could also using the `row_number() OVER` function native to sqlite3 to add a new column based on partitioning by `animals.zoo_id` and ordered by `animal.weight`.  Then we can filter on that row number.

In [17]:
func = sa.func.row_number().over(order_by=Animal.weight.desc(), partition_by=Animal.zoo_id)
func

In [18]:
label = func.label("row_number")
label

In [19]:
db.query(label).all()

2021-04-30 22:23:22,586 INFO sqlalchemy.engine.base.Engine SELECT row_number() OVER (PARTITION BY animals.zoo_id ORDER BY animals.weight DESC) AS row_number 
FROM animals
2021-04-30 22:23:22,589 INFO sqlalchemy.engine.base.Engine ()


[(1), (2), (3), (1), (2), (3), (1), (2), (3)]

In [20]:
db.query(Animal, label).all()

2021-04-30 22:23:22,801 INFO sqlalchemy.engine.base.Engine SELECT animals.id AS animals_id, animals.weight AS animals_weight, animals.zoo_id AS animals_zoo_id, row_number() OVER (PARTITION BY animals.zoo_id ORDER BY animals.weight DESC) AS row_number 
FROM animals
2021-04-30 22:23:22,802 INFO sqlalchemy.engine.base.Engine ()


[(Animal(id=1117, weight=1000, zoo_id=9), 1),
 (Animal(id=1415, weight=695, zoo_id=9), 2),
 (Animal(id=1410, weight=690, zoo_id=9), 3),
 (Animal(id=12993, weight=2190, zoo_id=11), 1),
 (Animal(id=9821, weight=750, zoo_id=11), 2),
 (Animal(id=96673, weight=580, zoo_id=11), 3),
 (Animal(id=99929, weight=1090, zoo_id=12), 1),
 (Animal(id=9992, weight=1040, zoo_id=12), 2),
 (Animal(id=1411, weight=690, zoo_id=12), 3)]

#### As a subquery

In [21]:
sq = db.query(Animal.id, sa.func.row_number().over(order_by=Animal.weight.desc(), partition_by=Animal.zoo_id).label("row_number")).subquery()
sq

<sqlalchemy.sql.selectable.Alias at 0x7f8a01438430; %(140230703416368 anon)s>

In [22]:
db.query(sq).all()

2021-04-30 22:23:23,110 INFO sqlalchemy.engine.base.Engine SELECT anon_1.id AS anon_1_id, anon_1.row_number AS anon_1_row_number 
FROM (SELECT animals.id AS id, row_number() OVER (PARTITION BY animals.zoo_id ORDER BY animals.weight DESC) AS row_number 
FROM animals) AS anon_1
2021-04-30 22:23:23,111 INFO sqlalchemy.engine.base.Engine ()


[(1117, 1),
 (1415, 2),
 (1410, 3),
 (12993, 1),
 (9821, 2),
 (96673, 3),
 (99929, 1),
 (9992, 2),
 (1411, 3)]

In [23]:
db.query(Animal).join(sq, Animal.id==sq.c.id).filter(sq.c.row_number==2).all()

2021-04-30 22:23:23,260 INFO sqlalchemy.engine.base.Engine SELECT animals.id AS animals_id, animals.weight AS animals_weight, animals.zoo_id AS animals_zoo_id 
FROM animals JOIN (SELECT animals.id AS id, row_number() OVER (PARTITION BY animals.zoo_id ORDER BY animals.weight DESC) AS row_number 
FROM animals) AS anon_1 ON animals.id = anon_1.id 
WHERE anon_1.row_number = ?
2021-04-30 22:23:23,261 INFO sqlalchemy.engine.base.Engine (2,)


[Animal(id=1415, weight=695, zoo_id=9),
 Animal(id=9821, weight=750, zoo_id=11),
 Animal(id=9992, weight=1040, zoo_id=12)]

#### As a CTE

In [24]:
cte = db.query(Animal.id, sa.func.row_number().over(order_by=Animal.weight.desc(), partition_by=Animal.zoo_id).label("row_number")).cte("add_row_number")
cte

<sqlalchemy.sql.selectable.CTE at 0x7f8a0143ca90; add_row_number>

In [25]:
db.query(cte).all()

2021-04-30 22:23:23,560 INFO sqlalchemy.engine.base.Engine WITH add_row_number AS 
(SELECT animals.id AS id, row_number() OVER (PARTITION BY animals.zoo_id ORDER BY animals.weight DESC) AS row_number 
FROM animals)
 SELECT add_row_number.id AS add_row_number_id, add_row_number.row_number AS add_row_number_row_number 
FROM add_row_number
2021-04-30 22:23:23,561 INFO sqlalchemy.engine.base.Engine ()


[(1117, 1),
 (1415, 2),
 (1410, 3),
 (12993, 1),
 (9821, 2),
 (96673, 3),
 (99929, 1),
 (9992, 2),
 (1411, 3)]

In [26]:
db.query(Animal).join(cte, Animal.id==cte.c.id).filter(cte.c.row_number==2).all()

2021-04-30 22:23:23,785 INFO sqlalchemy.engine.base.Engine WITH add_row_number AS 
(SELECT animals.id AS id, row_number() OVER (PARTITION BY animals.zoo_id ORDER BY animals.weight DESC) AS row_number 
FROM animals)
 SELECT animals.id AS animals_id, animals.weight AS animals_weight, animals.zoo_id AS animals_zoo_id 
FROM animals JOIN add_row_number ON animals.id = add_row_number.id 
WHERE add_row_number.row_number = ?
2021-04-30 22:23:23,786 INFO sqlalchemy.engine.base.Engine (2,)


[Animal(id=1415, weight=695, zoo_id=9),
 Animal(id=9821, weight=750, zoo_id=11),
 Animal(id=9992, weight=1040, zoo_id=12)]